In [1]:
from zipfile import ZipFile

archive = ZipFile("/content/data.zip")

archive.extractall()

archive.close()

Fazendo os imports

In [2]:

!pip install ultralytics -q

import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import yaml
from ultralytics import YOLO

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultraly

Definindo Diretorios

In [3]:


# Local original dos datasets
train_img_dir = "/content/kaggle/input/car-object-detection/data/training_images"
test_img_dir = "/content/kaggle/input/car-object-detection/data/testing_images"
labels_csv = "/content/kaggle/input/car-object-detection/data/train_solution_bounding_boxes.csv"



In [4]:
root_dir = "/kaggle/working/data"
os.makedirs(root_dir, exist_ok=True)
os.makedirs(f"{root_dir}/images/train", exist_ok=True)
os.makedirs(f"{root_dir}/images/val", exist_ok=True)
os.makedirs(f"{root_dir}/labels/train", exist_ok=True)
os.makedirs(f"{root_dir}/labels/val", exist_ok=True)

Convertendo CSV Annotations para o formato YOLO

In [5]:
df = pd.read_csv(labels_csv)
df['class'] = 0  # uma classe: carro
df.rename(columns={'image': 'img_name'}, inplace=True)

# Tamanho da imagem para o dataset
W, H = 676, 380

df["x_center"] = ((df["xmin"] + df["xmax"]) / 2) / W
df["y_center"] = ((df["ymin"] + df["ymax"]) / 2) / H
df["width"] = (df["xmax"] - df["xmin"]) / W
df["height"] = (df["ymax"] - df["ymin"]) / H

df_yolo = df[["img_name", "class", "x_center", "y_center", "width", "height"]]



Fazendo split de train e val e fazendo preparação do arquivo.

In [6]:
image_list = sorted(os.listdir(train_img_dir))
np.random.shuffle(image_list)

split = int(0.8 * len(image_list))
train_imgs = image_list[:split]
val_imgs = image_list[split:]

for img_name in image_list:
    subset = "train" if img_name in train_imgs else "val"

    if img_name in df_yolo["img_name"].values:
        rows = df_yolo[df_yolo["img_name"] == img_name]
        label_path = f"{root_dir}/labels/{subset}/{img_name.replace('.jpg','.txt')}"
        with open(label_path, 'w') as f:
            for _, row in rows.iterrows():
                f.write(f"{int(row['class'])} {row['x_center']} {row['y_center']} {row['width']} {row['height']}\n")

    # Copiando a imagem
    shutil.copy(f"{train_img_dir}/{img_name}", f"{root_dir}/images/{subset}/{img_name}")

 Ciando arquivo de configuração YAML

In [7]:
data_yaml = {
    'path': root_dir,
    'train': f"{root_dir}/images/train",
    'val': f"{root_dir}/images/val",
    'nc': 1,
    'names': ['car']
}

with open("/kaggle/working/yolo.yaml", 'w') as f:
    yaml.dump(data_yaml, f)

Classe para salvar tempos de epocas

In [8]:
import time

def on_train_epoch_start( trainer):

    with open("/content/epoch_times.txt", "a+") as f:
      f.write(f"Epoch {trainer.epoch + 1} started at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}\n")

def on_train_epoch_end( trainer):
    with open("/content/epoch_times.txt", "a+") as f:
      f.write(f"Epoch {trainer.epoch + 1} ended at {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}\n")

Treino YOLO

In [9]:
from datetime import datetime
model = YOLO("yolo11n.pt")

model.add_callback("on_train_epoch_start", on_train_epoch_start)
model.add_callback("on_train_epoch_end", on_train_epoch_end)

inicio_treino = datetime.now()

model.train(
    data="/kaggle/working/yolo.yaml",
    epochs=100,
    imgsz=640,
    batch=8,
    name="car_detector",
    patience=10
)

end_train = datetime.now()

with open("tempo_treino.txt", "w") as f:
    f.write(f"Tempo de treino: {end_train - inicio_treino}")
    f.close()

100%|██████████| 5.35M/5.35M [00:00<00:00, 97.4MB/s]


Ultralytics 8.3.137 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=car_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrai

100%|██████████| 755k/755k [00:00<00:00, 18.0MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,035 parameters, 2,590,019 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1974.3±1177.5 MB/s, size: 111.2 KB)


...

     92/100      2.53G     0.9407     0.4448      1.032          2        640: 100%|██████████| 100/100 [00:16<00:00,  5.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00, 10.76it/s]

                   all        201        127      0.991      0.992      0.995      0.698
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 82, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



92 epochs completed in 0.492 hours.
Optimizer stripped from runs/detect/car_detector/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/car_detector/weights/best.pt, 5.5MB

Validating runs/detect/car_detector/weights/best.pt...
Ultralytics 8.3.137 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:01<00:00,  7.05it/s]


                   all        201        127      0.988      0.984      0.994      0.709
Speed: 0.3ms preprocess, 2.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/car_detector


Evaluate Performance

In [11]:
model = YOLO("/content/runs/detect/car_detector/weights/best.pt")
metrics = model.val()
print("mAP@0.5:", metrics.box.map50)
print("mAP@0.5:0.95:", metrics.box.map)

Ultralytics 8.3.137 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1166.5±446.4 MB/s, size: 80.5 KB)


val: Scanning /kaggle/working/data/labels/val.cache... 77 images, 124 backgrounds, 0 corrupt: 100%|██████████| 201/201 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  5.42it/s]


                   all        201        127      0.995      0.984      0.995      0.708
Speed: 1.7ms preprocess, 4.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val
mAP@0.5: 0.9949212598425198
mAP@0.5:0.95: 0.7079153376081848


Inferencia nas imagens de teste

In [12]:
test_results = model.predict(source=test_img_dir, conf=0.5, iou=0.5, save=False)

# Salvando as predictions
pred_dir = "/kaggle/working/yolov11_results"
img_save_dir = f"{pred_dir}/images"
label_save_dir = f"{pred_dir}/labels"
os.makedirs(img_save_dir, exist_ok=True)
os.makedirs(label_save_dir, exist_ok=True)

for r in test_results:
    name = os.path.basename(r.path).split('.')[0]
    boxes = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()

    label_path = os.path.join(label_save_dir, f"{name}.txt")
    with open(label_path, "w") as f:
        for score, box in zip(confs, boxes):
            f.write(f"{score:.4f} " + " ".join(map(str, box)) + "\n")


image 1/175 /content/kaggle/input/car-object-detection/data/testing_images/vid_5_25100.jpg: 384x640 (no detections), 75.1ms
image 2/175 /content/kaggle/input/car-object-detection/data/testing_images/vid_5_25120.jpg: 384x640 (no detections), 12.1ms
image 3/175 /content/kaggle/input/car-object-detection/data/testing_images/vid_5_25140.jpg: 384x640 (no detections), 11.9ms
image 4/175 /content/kaggle/input/car-object-detection/data/testing_images/vid_5_25160.jpg: 384x640 (no detections), 16.4ms
image 5/175 /content/kaggle/input/car-object-detection/data/testing_images/vid_5_25180.jpg: 384x640 (no detections), 12.1ms
image 6/175 /content/kaggle/input/car-object-detection/data/testing_images/vid_5_25200.jpg: 384x640 (no detections), 11.2ms
image 7/175 /content/kaggle/input/car-object-detection/data/testing_images/vid_5_25220.jpg: 384x640 (no detections), 13.3ms
image 8/175 /content/kaggle/input/car-object-detection/data/testing_images/vid_5_25240.jpg: 384x640 (no detections), 15.4ms
image 9